In [51]:
import glob,os
from PIL import Image,ImageDraw,ImageFont

In [52]:
DIR = "/Users/damoncrockett/Desktop/global-bigbright/"
CROP = DIR+"cropped/"
SCALE = CROP+"scaled/"
LABEL = SCALE+"labeled/"
RESCALE = LABEL+"rescaled/"

In [53]:
files = glob.glob(os.path.join(DIR,"*.jpg"))

### Crop out Antarctica

In [54]:
for item in files:
    base = os.path.basename(item)
    im = Image.open(item)
    im = im.crop((220,100,4410,2070)) # derived from photoshop exploration
    im.save(CROP+base)

### Resize

In [55]:
files = glob.glob(os.path.join(CROP,"*.jpg"))

In [56]:
for item in files:
    base = os.path.basename(item)
    im = Image.open(item)
    im = im.resize((3840,1805),Image.ANTIALIAS)
    im.save(SCALE+base)

### New files

In [57]:
files = glob.glob(os.path.join(SCALE,"*.jpg"))

### Add labels

In [58]:
import pandas as pd 
from dateutil import parser
import datetime

def teparse(dt):
    try:
        return parser.parse(dt)
    except:
        return "err"

def pTsubset(df):
    try:
        return df[df.postedTime!="err"]
    except:
        return df

df = pd.read_csv("/Users/damoncrockett/Desktop/spacetime_2011.csv")
df.postedTime = df.postedTime.apply(teparse)
df = pTsubset(df)

t0 = df.postedTime.min()
delta = datetime.timedelta(days=7)
nweeks = 150 # fact about our data

weeklist = []
lbound = t0
for i in range(nweeks):
    rbound = lbound + delta
    weeklist.append((lbound, rbound))
    lbound = rbound

In [59]:
weekstarts = [item[0] for item in weeklist]

In [60]:
labellist = []
for item in weekstarts:
    year = str(item.year)
    month = str(item.month).zfill(2)
    labelstring = month+"."+year
    labellist.append(labelstring)

In [61]:
FONTDIR = "/Library/Fonts/"
font = ImageFont.truetype(FONTDIR+"DIN Alternate Bold.ttf",142)

In [62]:
for i in range(len(files)):
    pathstring = files[i]
    base = os.path.basename(pathstring)
    
    lbl = labellist[i]
    im = Image.open(pathstring)
    
    draw = ImageDraw.Draw(im)
    
    # month.year label
    draw.text((200,1500),text=lbl,font=font,fill="hsl(0,0%,25%)")
    
    # progress line
    draw.line([(200,1665),(200+i*3,1665)],fill="hsl(0,0%,25%)",width=9)
    
    im.save(LABEL+base)

### Re-scaling for smaller formats

In [63]:
files = glob.glob(os.path.join(LABEL,"*.jpg"))

In [64]:
for item in files:
    base = os.path.basename(item)
    im = Image.open(item)
    im = im.resize((1920,903),Image.ANTIALIAS)
    im.save(RESCALE+base)